En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

-----
#### Enfoque y lógica soluciones *q_time*

Para las soluciones *q_time* del challenge, el enfoque se centra en optimizar el tiempo de ejecución. Utilizado la biblioteca **Polars**, que es una biblioteca de manipulación de datos de alto rendimiento y baja memoria se realizó la lectura y procesamiento de los datos.

Polars utiliza una estructura de datos llamada LazyFrame que es similar a la de Pandas, pero utiliza un enfoque de evaluación perezosa (lazy evaluation), lo que significa que las operaciones no se ejecutan inmediatamente, sino que se almacenan en un plan de ejecución y sólo se realizan cuando se necesita el resultado. Esto puede llevar a un uso más eficiente de los recursos y a un mejor rendimiento.

**Mejoras al enfoque:** 
- Se podría utilizar servicios de almacenamiento en la nube como *Google Cloud Storage* o *Amazon S3* para almacenar el conjunto de datos. De esta forma la carga de los datos no pasan por la memoria de una sola máquina.

- Podríamos utilizar servicios de Big Data y análisis como *Google BigQuery* o *Amazon Redshift* para procesar el conjunto de datos. Estos servicios están diseñados para trabajar con grandes conjuntos de datos y pueden realizar operaciones de agrupamiento y conteo de manera muy rápida.

En general si se elige un enfoque utilizando herramientas en la nube podriamos aprovechar el conjunto de herramientas que ofrecen las diferentes suits como GCP, AWS, Azure y que interactuan de manera muy eficiente entre si.

#### Enfoque y lógica soluciones *q_memory*

Para las soluciones *q_memory* del challenge el enfoque se centra en optimizar el uso de la memoria. En lugar de cargar todos los datos en la memoria a la vez, se leen los datos en trozos (chunks) de 1000 filas a la vez utilizando la función *read_json* de **Pandas** con el parámetro *chunksize*.

En la inspección previa del archivo pudimos ver que son más de 117000 lineas de tweets, entonces utilizaremos chunks de tamaño 1000 para tener un balance entre un buen tiempo de ejecución y bajo consumo de memoría.

En cuanto al filtrado y procesamiento de los datos utilizamos las clases *Counter* y *defaultdict* de la biblioteca **Collections**, que proporcionan una forma eficiente de contar elementos.

Este enfoque es más eficiente en términos de uso de memoria, ya que sólo mantiene en memoria los contadores y el chunk actual de datos. Sin embargo, puede ser más lento que el enfoque q1_time, especialmente para conjuntos de datos muy grandes, ya que tiene que leer los datos en múltiples pasos.

**Mejoras al enfoque:**

- El uso de herramientas como **Apache Spark** en la nube son especialmente utiles para el procesamiento paralelo de grandes conjuntos de datos. Por ejemplo *Google Cloud Dataproc* (de GCP) o *Amazon EMR* de (AWS) son servicios permiten ejecutar trabajos de Apache Spark en la nube.

- *Paralelización:* Podríamos mejorar el rendimiento de este enfoque utilizando procesamiento paralelo o concurrente. Por ejemplo, podríamos procesar varios chunks de datos simultáneamente en diferentes hilos o procesos.

- Ajuste del tamaño del chunk: El tamaño del chunk seteado a 1000 fue elegido arbitrariamente. Podríamos experimentar con diferentes tamaños de chunk para encontrar el óptimo entre el tiempo de ejecución y el uso de memoria.


----
### Exploración de datos

Tras una inspección al conjunto de datos y las preguntas que se plantean en el desafío, podemos concluir que los campos que se utilizarán son los siguientes:


##### Índices que se utilizarán:

|Atributo|Descripción|Data type|Sub-elementos|
|--------|--------|--------|--------|
|date|Fecha cuando el tweet fue creado|String|-|
|user|Objeto con los datos del usuario que creó el tweet|User object|username (String)|
|content|Contenido del tweet|String|-|
|mentionedUsers|Lista de objetos con los datos de los usuarios mencionados en el tweet|List of User objects|username (String)|

##### Observaciones

- **content:** El elemento *content* que utilizarémos para la pregunta 2 del desafío contiene todo el contenido del tweet, incluyendo enlaces y menciones a usuarios, por lo que sería una buena practica realizar una limpieza de URLS y menciones antes de realizar el análisis.

- **created_at**: El atributo *created_at* que aparece en la documentación entregada para el desafió no está presente en el conjunto de datos, en su lugar está el atributo "date" que simboliza lo mismo, la fecha en la que se creó el tweet.

----
**Imports**

In [1]:
file_path = "../data/farmers-protest-tweets-2021-2-4.json"

%load_ext memory_profiler

#Funciones desarrolladas
from q1_time import q1_time, q1_time_pandas
from q1_memory import q1_memory
from q2_time import q2_time
from q2_memory import q2_memory
from q3_time import q3_time
from q3_memory import q3_memory

----
### Pregunta nro 1

1. Listar las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días.


##### **q1_time**

Para optimizar el tiempo de ejecución, cargamos el archivo completo en un *LazyFrame* de **Polars** y utilizamos los métodos de esta libreria ya que son eficientes para el procesamiento de datos ordenados.

El desglose de lo que realizó la función es:
- Seleccionar las columnas relevantes.
- Agrupamos los datos por por fecha y usuario.
- Contar el número de tweets para cada combinación de fecha y usuario.
- Seleccionar las 10 fechas con más tweets.
- Luego iteramos en estas fechas y seleccionar los usuarios con la mayor cantidad de tweets en esos días.
- Se contruye una lista de tuplas con la respuesta.

In [2]:
resultado_q1_time = q1_time(file_path)
display(resultado_q1_time)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

Medimos el tiempo que tardó nuestra función en encontrar el resultado y el peak máximo de memoría que utilizó

In [3]:
%timeit q1_time(file_path)
%memit q1_time(file_path)

2.83 s ± 87.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 1677.14 MiB, increment: 522.91 MiB


Con este enfoque podemos ver que la función **q1_time** tiene un tiempo de ejecución promedio de *2.8 segundos* y uso máximo de memoría de *1677 MiB*.

##### **q1_memory**

Para esta solución se leen los datos en chunks y aprovechando las iteraciones para cada chunk, actualiza dos contadores: uno para el conteo de tweets por fecha y otro para el conteo de tweets por usuario para cada fecha. Estos contadores se implementan utilizando las clases *Counter* y *defaultdict* de la biblioteca **Collections**, que proporcionan una forma eficiente de contar elementos.

El desglose de lo que realizó la función es:
- Definir el tamaño del chunk y los contadores.
- Leer el archivo JSON en chunks.
- Actualizar los contadores.
- Obtener las 10 fechas con más tweets.
- Seleccionar el usuario con más tweets para cada fecha.
- Construir una lista de tuplas con la respuesta.

In [4]:
resultado_q1_memory = q1_memory(file_path)
display(resultado_q1_memory)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

Medimos el tiempo que tardó nuestra función en encontrar el resultado y el peak máximo de memoría que utilizó

In [3]:
%timeit q1_memory(file_path)
%memit q1_memory(file_path)

4.42 s ± 175 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 157.44 MiB, increment: 15.60 MiB


Con este enfoque podemos ver que la función **q1_memory** tiene un tiempo de ejecución promedio de *4.4 segundos* y un uso máximo de memoría de *157 MiB*.

##### **q1_time_pandas**

Como extra mencionar que durante el desarrollo del challenge se intentó utilizar la librería pandas para los enfoques *time* enfoque que no cuida el uso de memoría, pero esta librería tardaba mucho tiempo en leer el conjunto de datos. Por lo que se barajó utilizar la librería **Polars** con su metodo LazyFrame, que es es más rápido que un DataFrame de Pandas gracias a su enfoque de procesamiento perezoso (lazy) que optimiza las operaciones y su capacidad de computación en paralelo aprovechando múltiples núcleos de CPU.

In [6]:
resultado_q1_time_pandas= q1_time_pandas(file_path)
display(resultado_q1_time_pandas)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

Medimos el tiempo que tardó nuestra función en encontrar el resultado y el peak máximo de memoría que utilizó

In [4]:
%timeit q1_time_pandas(file_path)
%memit q1_time_pandas(file_path)

5.11 s ± 583 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 2805.34 MiB, increment: 2319.15 MiB


Podemos ver que el uso de memoria y el tiempo de computo utilizando Pandas es mayor que utilizando la librería Polars. 5.11 segundos y 2805 MiB de memoría

----
### Pregunta nro 2

2. Los top 10 emojis más usados con su respectivo conteo.

##### **q2_time**

Para optimizar el tiempo de ejecución, cargamos el archivo completo en un *LazyFrame* de **Polars** y utilizamos los métodos de esta libreria ya que son eficientes para el procesamiento de datos ordenados.

Además antes de procesor el conjunto de datos en busca de los emojis haremos una limpieza para eliminar las URLs y las menciones de usuarios en el texto del tweet usando una expresión regular. Esto hará que la función *extract_emojis* tenga menos texto que procesar.

El desglose de lo que realizó la función es:
- Leer el conjunto de datos en un LazyFrame de Polars
- Eliminar los URLs y las menciones de usuario del texto del tweet usando una expresión regular.
- Seleccionar las columna relevante ("content") y aplica extract_emojis a cada fila (devuelve una lista de emojis de un texto).
- Filtra las filas del DataFrame para eliminar aquellas que no contienen emojis.
- Aplicamos la función explode() para tener un DataFrame donde cada fila corresponde a un solo emoji
- Agrupamos los datos por "emojis".
- Contar el número de veces que cada emoji aparece.
- Seleccionar los 10 emojis más usados.
- Se devuelve una lista de tuplas con la respuesta.

In [3]:
resultado_q2_time = q2_time(file_path)
display(resultado_q2_time)

[('🙏', 5046),
 ('😂', 3072),
 ('🚜', 2970),
 ('🌾', 2181),
 ('🇮🇳', 2086),
 ('🤣', 1665),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

Medimos el tiempo que tardó nuestra función en encontrar el resultado y el peak máximo de memoría que utilizó.

In [4]:
%timeit q2_time(file_path)
%memit q2_time(file_path)

7.23 s ± 220 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 565.34 MiB, increment: 340.73 MiB


Con este enfoque podemos ver que la función **q2_time** tiene un tiempo de ejecución promedio de *7.2 segundos* y uso máximo de memoría de *565 MiB*.

##### **q2_memory**

Para esta solución se leen los datos en chunks y aprovechando las iteraciones para cada chunk, actualiza dos contadores: uno para el conteo de tweets por fecha y otro para el conteo de tweets por usuario para cada fecha. Estos contadores se implementan utilizando las clases *Counter* y *defaultdict* de la biblioteca **Collections**, que proporcionan una forma eficiente de contar elementos.

El desglose de lo que realizó la función es:
- Definir el tamaño del chunk y crear un contador para los emojis (emoji_counts).
- Leer el archivo JSON en chunks.
- Usar la función *emoji_list* de la librería emoji para extraer los emojis en cada tweet
- Actualizar los contadores de emojis.
- Obtener los 10 emojis que más se usaron en los tweets desde el contador usando el método most_common(10).
- Construir una lista de tuplas con la respuesta.

In [4]:
resultado_q2_memory = q2_memory(file_path)
display(resultado_q2_memory)

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]

Medimos el tiempo que tardó nuestra función en encontrar el resultado y el peak máximo de memoría que utilizó.

In [2]:
%timeit q2_memory(file_path)
%memit q2_memory(file_path)

10.9 s ± 323 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 156.49 MiB, increment: 14.55 MiB


Con este enfoque podemos ver que la función **q2_memory** tiene un tiempo de ejecución promedio de *10.9 segundos* y uso máximo de memoría de *156 MiB*.

----
### Pregunta nro 3

3. El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos.

##### **q3_time**

Para optimizar el tiempo de ejecución, cargamos el archivo completo en un *LazyFrame* de **Polars** y utilizamos los métodos de esta libreria ya que son eficientes para el procesamiento de datos ordenados.

El desglose de lo que realizó la función es:
- Leer el conjunto de datos en un LazyFrame de Polars
- Explosión de la columna *mentionedUsers* para poder contar cada mención por separado.
- Extraemos el *username* de cada usuario mencionado y se almacenan en una nueva columna *username*.
- Agrupar los datos por la nueva columna *username* y se realiza el conteo de menciones.
- Ordenamos y obtenemos el top 10 de usuarios más mencionados con *limit(10)*.
- Se devuelve una lista de tuplas con la respuesta.

In [4]:
resultado_q3_time = q3_time(file_path)
display(resultado_q3_time)

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]

Medimos el tiempo que tardó nuestra función en encontrar el resultado y el peak máximo de memoría que utilizó.

In [2]:
%timeit q3_time(file_path)
%memit q3_time(file_path)

2.63 s ± 99.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 1410.25 MiB, increment: 804.50 MiB


Con este enfoque podemos ver que la función **q3_time** tiene un tiempo de ejecución promedio de *2.6 segundos* y uso máximo de memoría de *1410 MiB*.

##### **q3_memory**

Para esta solución se leen los datos en chunks y aprovechando las iteraciones para cada chunk, se actualiza un contador con las menciones de cada usuario. Este contador se implementa utilizando la clase *Counter* de la biblioteca **Collections**, que proporciona una forma eficiente de contar elementos.

El desglose de lo que realizó la función es:
- Definir el tamaño del chunk y crear el contador que almacenará las menciones (user_mention_counts).
- Leer el archivo JSON en chunks.
- Explosión de menciones para poder contar cada mención por separado y eliminamos filas que no tengan menciones.
- Extraemos el *username* de cada mención y lo almacenamos en una nueva columna llamada 'username'.
- Actualizar el contador.
- Obtenemos los 10 usuarios con más menciones desde el contador, con el método *most_common(10)*.
- Construir una lista de tuplas con la respuesta.

In [3]:
resultado_q3_memory = q3_memory(file_path)
display(resultado_q3_memory)

[('narendramodi', 2265),
 ('Kisanektamorcha', 1840),
 ('RakeshTikaitBKU', 1644),
 ('PMOIndia', 1427),
 ('RahulGandhi', 1146),
 ('GretaThunberg', 1048),
 ('RaviSinghKA', 1019),
 ('rihanna', 986),
 ('UNHumanRights', 962),
 ('meenaharris', 926)]

Medimos el tiempo que tardó nuestra función en encontrar el resultado y el peak máximo de memoría que utilizó.

In [2]:
%timeit q3_memory(file_path)
%memit q3_memory(file_path)

4.6 s ± 53.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
peak memory: 150.80 MiB, increment: 10.68 MiB


Con este enfoque podemos ver que la función **q3_memory** tiene un tiempo de ejecución promedio de *4.6 segundos* y uso máximo de memoría de *150 MiB*.

----
### Estructura del proyecto


Se siguió la misma estructura definida en el challenge, agregando una carpeta donde se almacenaron los datos

```css
├── README.md
├── requirements.txt
├── LICENSE
├── data
│   └── farmers-protest-tweets-2021-2-4.json
└── src
    ├── challenge.ipynb
    ├── q1_time.py
    ├── q1_memory.py
    ├── q2_time.py
    ├── q2_memory.py
    ├── q3_time.py
    ├── q3_memory.py
    └── utils.py
   ```

----
### POST para enviar el desafío

In [2]:
import requests

data = {
   "name": "Victor Gatica",
   "mail": "gaticavm9@gmail.com",
   "github_url": "https://github.com/gaticavm9/DE-Challenge-Victor-Gatica.git"
}

response = requests.post("https://advana-challenge-check-api-cr-k4hdbggvoq-uc.a.run.app/data-engineer", json=data)

# Verifica que la solicitud se haya realizado correctamente
if response.status_code == 200:
   print("Desafío enviado correctamente.")
else:
   print(f"Hubo error: {response.content}")

Desafío enviado correctamente.
